In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np

In [7]:
# Memastikan penggunaan GPU jika tersedia
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Device Name:", torch.cuda.get_device_name(0))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

CUDA Available: True
GPU Device Name: NVIDIA GeForce GTX 1050
Using device: cuda


In [8]:
# Transformations for CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
# Define CNN architecture
class CNN(nn.Module):
    def __init__(self, kernel_size=3, pooling='max'):
        super(CNN, self).__init__()
        
        if pooling == 'max':
            self.pool = nn.MaxPool2d(2, 2)
        elif pooling == 'avg':
            self.pool = nn.AvgPool2d(2, 2)

        self.conv1 = nn.Conv2d(3, 32, kernel_size=kernel_size, padding=kernel_size//2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=kernel_size//2)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Training function with Early Stopping trigger
def train_model(model, optimizer, criterion, scheduler, num_epochs=5, early_stopping_patience=10):
    train_losses = []
    val_losses = []
    best_loss = float('inf')
    patience = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in trainloader:
            inputs, labels = inputs.cuda(), labels.cuda()

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        train_loss = running_loss / len(trainloader)
        train_losses.append(train_loss)

        # Validation loss
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for inputs, labels in testloader:
                inputs, labels = inputs.cuda(), labels.cuda()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        val_loss /= len(testloader)
        val_losses.append(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Early stopping trigger logic
        if val_loss < best_loss:
            best_loss = val_loss
            patience = 0
        else:
            patience += 1
            if patience >= early_stopping_patience:
                print("Early stopping triggered due to no improvement in validation loss.")
                break

        # Step the scheduler
        scheduler.step(val_loss)

    return train_losses, val_losses

# Compare models
kernel_sizes = [3, 5, 7]
pooling_methods = ['max', 'avg']
optimizers = {'SGD': optim.SGD, 'RMSProp': optim.RMSprop, 'Adam': optim.Adam}
num_epochs = [5, 50, 100, 250, 350]

results = {}

for kernel_size in kernel_sizes:
    for pooling in pooling_methods:
        for opt_name, opt_fn in optimizers.items():
            for epoch in num_epochs:
                print(f"Kernel: {kernel_size}, Pooling: {pooling}, Optimizer: {opt_name}, Epochs: {epoch}")

                model = CNN(kernel_size=kernel_size, pooling=pooling).cuda()
                criterion = nn.CrossEntropyLoss()
                optimizer = opt_fn(model.parameters(), lr=0.01)
                scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

                train_losses, val_losses = train_model(
                    model, optimizer, criterion, scheduler, num_epochs=epoch, early_stopping_patience=10
                )

                # Save results
                results[(kernel_size, pooling, opt_name, epoch)] = (train_losses, val_losses)

# Save results to a file
np.save('cnn_cifar10_results.npy', results)
print("All experiments completed.")


Kernel: 3, Pooling: max, Optimizer: SGD, Epochs: 5
Epoch 1/5, Train Loss: 2.1633, Val Loss: 1.9789
Epoch 2/5, Train Loss: 1.8674, Val Loss: 1.7616
Epoch 3/5, Train Loss: 1.7046, Val Loss: 1.6486
Epoch 4/5, Train Loss: 1.5757, Val Loss: 1.5355
Epoch 5/5, Train Loss: 1.4698, Val Loss: 1.4284
Kernel: 3, Pooling: max, Optimizer: SGD, Epochs: 50
Epoch 1/50, Train Loss: 2.1679, Val Loss: 1.9862
Epoch 2/50, Train Loss: 1.8677, Val Loss: 1.7552
Epoch 3/50, Train Loss: 1.6901, Val Loss: 1.6096
Epoch 4/50, Train Loss: 1.5716, Val Loss: 1.5288
Epoch 5/50, Train Loss: 1.4736, Val Loss: 1.4244
Epoch 6/50, Train Loss: 1.3929, Val Loss: 1.3661
Epoch 7/50, Train Loss: 1.3351, Val Loss: 1.3448
Epoch 8/50, Train Loss: 1.2855, Val Loss: 1.2562
Epoch 9/50, Train Loss: 1.2450, Val Loss: 1.2416
Epoch 10/50, Train Loss: 1.2094, Val Loss: 1.2308
Epoch 11/50, Train Loss: 1.1746, Val Loss: 1.1725
Epoch 12/50, Train Loss: 1.1450, Val Loss: 1.1656
Epoch 13/50, Train Loss: 1.1127, Val Loss: 1.1213
Epoch 14/50, Tra